#**MEDICAMENTOS VITALES NO DISPONIBLES**


*   Sofia Gomez Rodriguez
*   Victor Andres Martinez Preciado
*   Juan Esteban Paredes Alvarez
*   Sofia Reyes Molina




## 1). Busquen una fuente de datos de tipo estructurados o semi que potencialmente aporte a los ODS.

###The following structured data source was found: https://www.datos.gov.co/Salud-y-Protecci-n-Social/MEDICAMENTOS-VITALES-NO-DISPONIBLES/sdmr-tfmf/about_data

###The dataset found is considered to be directly related to ODS number 3, health and well-being, where the context is about essential and irreplaceable medicines to safeguard the life or alleviate the suffering of a patient or group of patients, which are not available in the country or are not available in sufficient quantities due to low-cost marketing conditions.


##2). Realicen la extracción de datos

In [ ]:
from google.colab import drive

#Connect to Drive
drive.mount('/content/drive')

# Define the base path where the files are located
ruta = "/content/drive/MyDrive/ETL/MEDICAMENTOS_VITALES_NO_DISPONIBLES_20260202.csv"

Mounted at /content/drive


In [ ]:
import pandas as pd

# Read a CSV file delimited by ','
df = pd.read_csv(ruta)
df.head(5)

,FECHA_DE_AUTORIZACIÓN,TIPO_DE_SOLICITUD,SOLICITANTE/IMPORTADOR,IUM,PRINCIPIO_ACTIVO1,CONCENTRACIÓN_DELMEDICAMENTO1,UNIDAD_MEDIDA1,PRINCIPIO_ACTIVO2,CONCENTRACIÓN_DEL_MEDICAMENTO2,UNIDAD_MEDIDA2,FORMA_FARMACÉUTICA,NOMBRE_COMERCIAL_,CANTIDAD_SOLICITADA,PRESENTACIÓN_COMERCIAL,DIAGNOSTICO_CIE-1NO REPORTA,CÓDIGO_DIAGNOSTICO_CIE-10
0,2026 Jan 14 12:00:00 AM,MÁS DE UN PACIENTE,GLOBAL SERVICE PHARMACEUTICAL S.A.S.,1E1023551001101,EPINEFRINA,0.5,mg/mL,NO APLICA,NO APLICA,NO APLICA,SOLUCIÓN INYECTABLE,PENEPIN JUNIOR,10.0,"CAJA X 2 APLICADORES X 0,3 ML",ANAFILAXIA,NO REPORTADO
1,2026 Jan 14 12:00:00 AM,MÁS DE UN PACIENTE,GLOBAL SERVICE PHARMACEUTICAL S.A.S.,1E1005681005101,EPINEFRINA,1,mg/mL,NO APLICA,NO APLICA,NO APLICA,SOLUCIÓN INYECTABLE,PENEPIN,28.0,"CAJA X 2 APLICADORES X 0,3 ML",ANAFILAXIA,NO REPORTADO
2,2026 Jan 14 12:00:00 AM,PACIENTE ESPECIFICO,VALENTECH PHARMA COLOMBIA SAS,1E1029551000102,ETEPLIRSEN,50,mg/mL,NO APLICA,NO APLICA,NO APLICA,SOLUCIÓN INYECTABLE,EXONDYS 51,96.0,CAJAS X 1 VIAL X 2 mL,DISTROFIA MUSCULAR,G710
3,2026 Jan 14 12:00:00 AM,PACIENTE ESPECIFICO,VALENTECH PHARMA COLOMBIA SAS,1E1029551000101,ETEPLIRSEN,50,mg/mL,NO APLICA,NO APLICA,NO APLICA,SOLUCIÓN INYECTABLE,EXONDYS 51,24.0,CAJAS X 1 VIAL X 10 mL,DISTROFIA MUSCULAR,G710
4,2026 Jan 14 12:00:00 AM,PACIENTE ESPECIFICO,GLOBAL SERVICE PHARMACEUTICAL S.A.S.,1A1001611000100,AMILORIDE,5,mg,NO APLICA,NO APLICA,NO APLICA,TABLETA,MODAMIDE,18.0,CAJAS X 30 TABLETAS,SINDROME DE GITELMAN,N158


## 3). ¿Esta fuente de datos ODS debería cargarse mediante una estrategia de batch o streaming?
###It should be loaded using a batch strategy since a large volume of data is accumulated and it is loaded from a CSV file rather than being processed as it arrives.

## 4) ¿Sería una carga completa o incremental?
###It is a full load because the data is brought from the data source only once, without checking for updates.

## 5) Realicen una exploración del conjunto de datos (revisar nulos, vacios, tamaño del dataset, diccionario, detección de anomalias)

##Dictionary:
https://www.datos.gov.co/Salud-y-Protecci-n-Social/MEDICAMENTOS-VITALES-NO-DISPONIBLES/sdmr-tfmf/about_data

In [ ]:
# Show general information about the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9711 entries, 0 to 9710
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   FECHA_DE_AUTORIZACIÓN           9711 non-null   object 
 1   TIPO_DE_SOLICITUD               9711 non-null   object 
 2   SOLICITANTE/IMPORTADOR          9708 non-null   object 
 3   IUM                             9708 non-null   object 
 4   PRINCIPIO_ACTIVO1               9711 non-null   object 
 5   CONCENTRACIÓN_DELMEDICAMENTO1   9672 non-null   object 
 6   UNIDAD_MEDIDA1                  9612 non-null   object 
 7   PRINCIPIO_ACTIVO2               9690 non-null   object 
 8   CONCENTRACIÓN_DEL_MEDICAMENTO2  9692 non-null   object 
 9   UNIDAD_MEDIDA2                  9690 non-null   object 
 10  FORMA_FARMACÉUTICA              9674 non-null   object 
 11  NOMBRE_COMERCIAL_               9710 non-null   object 
 12  CANTIDAD_SOLICITADA             96

In [ ]:
# This allows you to observe the number of null values per column

serie_1 = df.isna().sum()

for column in serie_1.index:
  por =  100*serie_1[column]/df.shape[0]
  print(f"El porcentaje de nulos en la columna {column}es: {por:.2f}%")

El porcentaje de nulos en la columna FECHA_DE_AUTORIZACIÓNes: 0.00%
El porcentaje de nulos en la columna TIPO_DE_SOLICITUDes: 0.00%
El porcentaje de nulos en la columna SOLICITANTE/IMPORTADORes: 0.03%
El porcentaje de nulos en la columna IUMes: 0.03%
El porcentaje de nulos en la columna PRINCIPIO_ACTIVO1es: 0.00%
El porcentaje de nulos en la columna CONCENTRACIÓN_DELMEDICAMENTO1es: 0.40%
El porcentaje de nulos en la columna UNIDAD_MEDIDA1es: 1.02%
El porcentaje de nulos en la columna PRINCIPIO_ACTIVO2es: 0.22%
El porcentaje de nulos en la columna CONCENTRACIÓN_DEL_MEDICAMENTO2es: 0.20%
El porcentaje de nulos en la columna UNIDAD_MEDIDA2es: 0.22%
El porcentaje de nulos en la columna FORMA_FARMACÉUTICAes: 0.38%
El porcentaje de nulos en la columna NOMBRE_COMERCIAL_es: 0.01%
El porcentaje de nulos en la columna CANTIDAD_SOLICITADAes: 0.61%
El porcentaje de nulos en la columna PRESENTACIÓN_COMERCIALes: 0.38%
El porcentaje de nulos en la columna DIAGNOSTICO_CIE-1NO REPORTAes: 0.00%
El porce

In [ ]:
# Replace empty spaces with null values.
import numpy as np

df = df.replace(r'^\s*$', np.nan, regex=True)

In [ ]:
# Size of the dataset

print(f"El numero de filas que tiene es de {df.shape[0]} y {df.shape[1]} columnas")

El numero de filas que tiene es de 9711 y 16 columnas


In [ ]:
### Identify duplicate IUM records

df["ID_Duplicado"] = df.duplicated(subset=["IUM"], keep=False)

##6). Identificar si la fuente de datos requiere un pre-procesamiento para las siguientes etapas

###The preprocessing steps for the following stages would include verifying the following:

###- Number of negative requests

###- Negative medication concentration values

###- Very old dates

###- Ensuring all dates are in the same format

###- Converting all data fields to their appropriate data types (for example, converting the medication concentration column to float64)

###- Converting the request date to datetime



##7.) Sugerir nuevas fuentes que puedan contextualizar o potenciar los datos ya extraídos
###As new data sources, we can suggest datasets related to high medication prices and mortality caused by the lack of access to vital medicines.

### It would be useful to have the dictionary from the last column of the dataset CODIGO_DIAGNOSTICO_CIE-10, in order to have a better context on the use of each drug and its possible shortage.

# Ponerle numero al tipo de solicitud

In [ ]:
df['TIPO_DE_SOLICITUD'].unique()

array(['MÁS DE UN PACIENTE', 'PACIENTE ESPECIFICO', 'URGENCIA CLÍNICA'],
      dtype=object)

In [ ]:
df['UNIDAD_MEDIDA1'].unique()

array(['mg/mL', 'mg', 'MUI/mL', 'mg/Vial', 'DDP/ml', 'mg/g', 'µg/ml',
       'mcg/mL', 'g', 'mEq', 'HEP/mL', 'mcg/dosis', 'UI/mL', nan, 'MBq',
       'mEq/ml', 'UI', 'mcg', 'UT/mL', 'UI/Vial', '%', 'µg/Dosis (1ml)',
       'VARIAS', 'g/ml', 'mcg/ml (1 dosis)', 'ml/ml', 'HEP', 'MUI',
       'mg/U', 'PFU/vial (Mínimo)', 'µg prot / ml', 'mcg/U', 'VARIOS',
       'mg/1ml', 'ml/U', 'g/L', 'mcg/g', 'm3', 'MUI/vial', 'Dosis/ml',
       'mL', 'TU/ml', 'DL50', 'mCi', 'g/5ml', 'UB/mL', 'GBq / vial',
       'mg / Vial que corresponde a\n(27 mg / Vial) Peso Seco',
       'MBq / vial', 'GBq', 'GBq/ml', 'U'], dtype=object)

In [ ]:
df['UNIDAD_MEDIDA2'].unique()

array(['NO APLICA', 'mg/ml', 'mcg/ml', nan, 'mg', 'µg/ml', 'mg/mL', 'mcg',
       'g', '%'], dtype=object)

In [ ]:
nuevo_df = df[df["UNIDAD_MEDIDA1"] =="ml/ml"]
nuevo_df

,FECHA_DE_AUTORIZACIÓN,TIPO_DE_SOLICITUD,SOLICITANTE/IMPORTADOR,IUM,PRINCIPIO_ACTIVO1,CONCENTRACIÓN_DELMEDICAMENTO1,UNIDAD_MEDIDA1,PRINCIPIO_ACTIVO2,CONCENTRACIÓN_DEL_MEDICAMENTO2,UNIDAD_MEDIDA2,FORMA_FARMACÉUTICA,NOMBRE_COMERCIAL_,CANTIDAD_SOLICITADA,PRESENTACIÓN_COMERCIAL,DIAGNOSTICO_CIE-1NO REPORTA,CÓDIGO_DIAGNOSTICO_CIE-10,ID_Duplicado
1425,2025 Mar 27 12:00:00 AM,MÁS DE UN PACIENTE,INNOVEX INTERNATIONAL SAS,1A1061031000100,AMILO NITRITO,0.98,ml/ml,NO APLICA,NO APLICA,NO APLICA,SOLUCION NASAL,TROIKAA,10.0,CAJAS X 12 CARTUCHOS X 0.3 mL,NO REPORTADO,NO REPORTADO,True
3162,2024 Mar 12 12:00:00 AM,MÁS DE UN PACIENTE,INNOVEX INTERNATIONAL SAS,1A1061031000100,AMILO NITRITO,0.98,ml/ml,NO APLICA,NO APLICA,NO APLICA,SOLUCION NASAL,TROIKAA,10.0,CAJAS X 12 CARTUCHOS X 0.3 mL,ANTIDOTO CONTRA CIANURO,NO REPORTADO,True


In [ ]:
df['FORMA_FARMACÉUTICA'].unique()

array(['SOLUCIÓN INYECTABLE', 'TABLETA',
       'POLVO GRANULADO PARA RECONSTITUIR A SUSPENSIÓN ORAL',
       'POLVO PARA NEBULIZAR', 'SOLUCION PARA INFUSION',
       'POLVO LIOFILIZADO PARA RECONSTITUIR A SOLUCIÓN INYECTABLE',
       'CAPSULA', 'SUSPENSION INYECTABLE', 'JARABE',
       'GRANULOS DE LIBERACION NO MODIFICADA',
       'CAPSULAS DE LIBERACIÓN NO MODIFICADA', 'GEL',
       'POLVO PARA RECONSTITUIR PARA SOLUCIÓN PARA PERFUSIÓN',
       'POLVOS PARA NO RECONSTITUIR',
       'POLVO LIOFILIZADO PARA RECONSTITUIR A SOLUCIÓN INYECTABLE + DILUYENTE',
       'POLVO LIOFILIZADO PARA RECONSTITUCIÓN',
       'CAPSULAS DE LIBERACIÓN MODIFICADA',
       'POLVO PARA RECONSTITUIR A SUSPENSIÓN ORAL', 'POLVO LIOFILIZADO',
       'SOLUCIÓN CONCENTRADA PARA PERFUSIÓN',
       'TABLETA DE LIBERACION NO MODIFICADA', 'SUSPENSIÓN', 'CAPSULAS',
       nan, 'SOLUCIÓN ORAL', 'SUSPENSION ORAL',
       'POLVOS PARA RECONSTITUIR', 'SOLUCION',
       'TABLETAS DE LIBERACION MODIFICADA',
       'POLVO P